In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib qt

In [2]:
from scipy.optimize import curve_fit
import tkinter as tk
from tkinter import filedialog

matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})
%matplotlib qt

In [3]:
def gaussian(x, A, mu, sig):
    return A*exp(-power(x - mu, 2.) / (2 * power(sig, 2.)))

In [4]:
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# sélection des fichiers (Python 3)
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(initialdir = '/home/adrian/Documents/Mesures',multiple=True)
print(file_path)

# ('/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_000.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_001.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_002.dat')

('/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_000.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_001.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_002.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_003.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_004.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_005.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_006.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_007.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_008.dat')


In [ ]:
('/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_000.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_001.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_002.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_003.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_004.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_005.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_006.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_007.dat', '/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_008.dat')

In [6]:
# Array rectangle
# Allocation à l'avance

filename=file_path
m = loadtxt(file_path[1],usecols=(2),delimiter='\t',skiprows=0,unpack=True).size
n = len(file_path)
shape = (m, n)

TIME = zeros(shape)
PM = zeros(shape)
CH2 = zeros(shape)

for k in range(0,len(filename)):
    TIME[:,k], PM[:,k] = loadtxt(file_path[k],usecols=(0,1),delimiter='\t',skiprows=0,unpack=True)

In [7]:
fond = 720
PM = PM - fond

In [8]:
n = 1.0002746
c = 299792458
l866_1 = c/345.9993e12/n
l866_2 = c/346.00085e12/n
print(l866_1*1e9,l866_2*1e9)

l_866 = around(linspace(l866_2*1e9,l866_1*1e9,len(TIME[:,0])),5) # nm
f_866 = c/(n*l_866*1e-9) # Hz

866.2160807571738 866.212200318946


In [9]:
test_gauss = gaussian(TIME[:,0],1500,15,3)
fit_PM = zeros((len(file_path),3))
fit_PM = zeros((len(file_path),3))
covar_PM = [zeros((3,3)) for k in range(0,len(file_path))]

A = zeros(len(file_path))
mu = zeros(len(file_path))
sig = zeros(len(file_path))
            
fit_gauss = zeros((len(TIME[:,0]),(len(file_path))))

start = 0

In [10]:
import matplotlib.patches as patches

In [11]:
f_866

array([3.4600085e+14, 3.4600085e+14, 3.4600085e+14, ..., 3.4599930e+14,
       3.4599930e+14, 3.4599930e+14])

In [29]:
# plot with complicated scan plot only crystal part at the end

to_plot = 1
print(file_path[to_plot])
f_346 = 346000.234867

start = 0
stop = -1

start_exclude = 975
stop_exclude = 1150
x = f_866*1e-9-f_346

tata = (x[::-1]) # Ghz
x_fit = linspace(tata[0],tata[-1],5000)

xx = concatenate(( tata[:start_exclude],tata[stop_exclude:] ))
PMPM = concatenate(( PM[:start_exclude,to_plot],PM[stop_exclude:,to_plot] ))

bckg = min(PMPM)
fit_PM[to_plot,:], covar_PM[to_plot] = curve_fit(gaussian,xx,PMPM-bckg,
                                                    p0=[2500,0,0.5],maxfev = 50000)

A[to_plot] = fit_PM[to_plot,0]
mu[to_plot] = fit_PM[to_plot,1]
sig[to_plot] = fit_PM[to_plot,2]
FWHM = 2*sqrt(2*log(2)) * sig[to_plot] * 1000 # MHz
temperature = 40 * (FWHM*1e-3/f_346/(7.16*1e-7))**2

print(A[to_plot],mu[to_plot],sig[to_plot])
# fit_gauss[:,to_plot] = gaussian(x,A[to_plot],mu[to_plot],sig[to_plot])
my_fit_gauss = gaussian(x_fit,A[to_plot],mu[to_plot],sig[to_plot])

fig = figure('One res_noire',clear='True')
ax = fig.add_subplot(1, 1, 1)

# plot(l_866,PM[:,to_plot],'+',color='xkcd:prussian blue')
ax.plot(tata*1e3,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.')
# ax.plot(xx,PMPM,'4',color='k',label='cloud fluo.')
ax.plot(x_fit*1e3,my_fit_gauss+bckg,color='xkcd:vermillion',label='Gaussian fit')
# vlines(mu[to_plot]*1000,0,15000,linestyles=':',color='xkcd:vermillion')    

rect = patches.Rectangle((tata[start_exclude]*1e3, 0), (tata[stop_exclude]*1e3-tata[start_exclude]*1e3), 16000,
                         linewidth=1, edgecolor='black', facecolor='gray',alpha=.3,hatch='xxxx',label='fit exclusion')
# Add the patch to the Axes
ax.add_patch(rect)

    
text(-0.83*1e3, 8000, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.0f}$ MHz\n        T = {temperature:.0f} K',
rotation=0,
horizontalalignment='left',
verticalalignment='bottom',
color='xkcd:vermillion',
fontsize=17,
     bbox=dict(facecolor='white'))

# ax.set_xlim(-1614.37,-1613.55)
# ax.set_ylim(2800,15200)
xlabel(r'$\Delta \nu_{866}$ [MHz]')
ylabel('fluo. [cts/5ms]')
ax.legend(fontsize=18)

grid()
tight_layout()

savefig('866scan.png',dpi=300)

/home/adrian/Documents/Mesures/866_scan/20190603/scan_866_1V_1.5V_001.dat
6850.162356632052 -0.03996521165859502 0.23146460524776363


In [8]:
f_346 = 346000.234867
FWHM = 1.75*1e3
temperature = 40 * (FWHM*1e-3/f_346/(7.16*1e-7))**2
print(temperature)

1995.9849250532054
